In [16]:
from transformers import AutoFeatureExtractor, Wav2Vec2BertModel

processor = AutoFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")
# model = Wav2Vec2BertModel.from_pretrained("facebook/w2v-bert-2.0")

In [2]:
processor.sampling_rate

16000

In [17]:
import torchaudio

waveform, sampling_rate = torchaudio.load("ennuref.wav")
print(waveform.shape)
waveform = torchaudio.functional.resample(waveform, orig_freq=sampling_rate, new_freq=16000)
print(waveform.shape)
inputs = processor(waveform.numpy()[0], sampling_rate=16000, return_tensors="pt")
# outputs = model(**inputs)
inputs

torch.Size([2, 271250])
torch.Size([2, 98413])


{'input_features': tensor([[[-2.7274, -2.6149, -2.5163,  ..., -2.1149, -2.1674, -2.1311],
         [-2.0149, -1.9489, -1.7825,  ..., -1.7384, -1.8621, -1.6986],
         [-1.6377, -1.3883, -1.1396,  ..., -1.7544, -1.7077, -1.7296],
         ...,
         [-1.0482, -0.8214, -0.3950,  ..., -1.4695, -1.6222, -1.6071],
         [-1.1254, -0.7901, -0.4701,  ..., -1.6016, -1.6721, -1.5878],
         [-0.9931, -0.6387, -0.3397,  ...,  1.0000,  1.0000,  1.0000]]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1

In [3]:
outputs[0].shape

torch.Size([1, 307, 1024])

In [5]:
inputs.input_features.shape

torch.Size([1, 188, 160])

In [45]:
inputs.input_features.shape

torch.Size([1, 307, 160])

In [2]:
from languagecodec_decoder.feature_extractors import EncodecFeatures
import torch
import yaml

with open("checkpoints/pretrained_language_codec/config.yaml", "r") as f:
    config = yaml.safe_load(f)

# model = EncodecFeatures(**config['model']['init_args']["feature_extractor"]["init_args"])

# state_dict = torch.load("checkpoints/pretrained_language_codec/languagecodec_paper.ckpt", map_location="cpu")['state_dict']

# feature_extractor_state_dict = {}
# for k, v in state_dict.items():
#     if k.startswith('feature_extractor.'):
#         feature_extractor_state_dict[k.replace('feature_extractor.', '')] = v

# model.load_state_dict(feature_extractor_state_dict)

# del state_dict, feature_extractor_state_dict

In [4]:
config['model']['init_args']["backbone"]["init_args"]["input_channels"]

128

/Users/anhnguyenhoang/Desktop/vscode/speech-language-foundation/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [1]:

from languagecodec_encoder.utils import convert_audio
import torchaudio
import torch
from languagecodec_decoder.pretrained import Vocos

device=torch.device('cpu')

config_path = "checkpoints/pretrained_language_codec/config.yaml"
model_path = "checkpoints/pretrained_language_codec/model.ckpt"
languagecodec = Vocos.from_pretrained0802(config_path, model_path)
languagecodec = languagecodec.to(device)


/Users/anhnguyenhoang/Desktop/vscode/speech-language-foundation/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/Users/anhnguyenhoang/Desktop/vscode/speech-language-foundation/languagecodec_decoder/pretrained.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlist

In [2]:
wav, sr = torchaudio.load("ref_en_nu.wav")
wav = convert_audio(wav, sr, 24000, 1) 
bandwidth_id = torch.tensor([0])
wav=wav.to(device)
_,discrete_code= languagecodec.encode_infer(torch.cat([wav, wav]), bandwidth_id=bandwidth_id)
_.shape

torch.Size([2, 90837])


torch.Size([2, 128, 284])

In [3]:
languagecodec.feature_extractor.training

False

In [16]:
_,discrete_code= languagecodec.encode_infer(torch.cat([wav, wav]), bandwidth_id=bandwidth_id)
print(discrete_code.shape)
_.shape


torch.Size([8, 2, 462])


torch.Size([2, 128, 462])

In [18]:
print(discrete_code.shape)

torch.Size([8, 2, 462])


In [6]:
_.shape

torch.Size([1, 128, 462])

In [19]:
features = languagecodec.codes_to_features(discrete_code)
features.shape

torch.Size([2, 128, 462])

In [21]:
_[0,0,:10]

tensor([ 0.2876, -0.0315,  0.1397,  0.2557, -0.0260,  0.1442, -0.0943,  0.0470,
        -0.4271,  0.3227])

In [20]:
features[0,0,:10]

tensor([ 0.2876, -0.0315,  0.1397,  0.2557, -0.0260,  0.1442, -0.0943,  0.0470,
        -0.4271,  0.3227])

In [3]:
embed = torch.nn.Embedding(5, 16)

embed(torch.tensor([[1,2],[3,4]])).mean()

tensor(-0.0139, grad_fn=<MeanBackward0>)

In [4]:
from einops import rearrange

x = torch.tensor([[[1,2],[3,4]], [[1,2],[3,4]], [[1,2],[3,4]]])

x

tensor([[[1, 2],
         [3, 4]],

        [[1, 2],
         [3, 4]],

        [[1, 2],
         [3, 4]]])

In [9]:
x.transpose(1,2).shape

torch.Size([3, 2, 2])

In [10]:
x.shape

torch.Size([3, 2, 2])

In [2]:
import torch

x = torch.tensor([0.5, 0.5])

x.requires_grad

False

In [3]:
from transformers import EncodecModel, AutoProcessor
from models.codec_encoder import EncodecEncoderQuantizer

# model = EncodecEncoderQuantizer.from_pretrained("facebook/encodec_24khz")

# model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

In [2]:
processor

EncodecFeatureExtractor {
  "chunk_length_s": null,
  "feature_extractor_type": "EncodecFeatureExtractor",
  "feature_size": 1,
  "overlap": null,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 24000
}

In [8]:
import torchaudio

waveform, sampling_rate = torchaudio.load("ennuref.wav")
print(sampling_rate)
print(waveform.shape)
waveform = torchaudio.functional.resample(waveform, orig_freq=sampling_rate, new_freq=processor.sampling_rate)
print(waveform.shape)
inputs = processor([waveform.numpy()[0], waveform.numpy()[0][:-5000], waveform.numpy()[0][:-500]], sampling_rate=processor.sampling_rate, return_tensors="pt")
# audio_embed, codes_mask, codes = model(**inputs)

44100
torch.Size([2, 271250])
torch.Size([2, 147620])


In [14]:
x = {}
x.update(inputs)
x

{'padding_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32),
 'input_values': tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.4510e-04,
           1.4499e-04, 5.9232e-06]],
 
         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]]])}

In [21]:
hidden_states = audio_embed*codes_mask

print(codes_mask.shape)
print(hidden_states.shape)

pooled_output = hidden_states.sum(dim=1) / codes_mask.sum(dim=1) # B, I

pooled_output.shape

torch.Size([3, 462, 128])
torch.Size([3, 462, 128])


torch.Size([3, 128])

In [23]:
codes_mask.sum(dim=1)

tensor([[462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462., 462.,
         462., 462., 462., 462., 462., 462., 462., 462.],
        [449., 449., 449., 449., 449., 449., 449., 449., 449., 449., 449., 449.,
         449., 449., 449., 449., 449., 449., 449., 

In [30]:
import torch
import numpy as np
from transformers.trainer_utils import PredictionOutput

PredictionOutput(predictions=np.array([1,2,3]), label_ids=np.array([1,2,2]), metrics={"a": 1.0}).metrics

{'a': 1.0}

In [4]:
audio_embed.shape

torch.Size([3, 462, 128])

In [3]:
audio_embed*codes_mask

tensor([[[-0.7727,  8.4915, -3.5145,  ..., -0.6387, -1.6899,  1.7921],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         ...,
         [-0.5899,  8.6741, -3.5319,  ..., -0.5725, -1.6565,  1.6399],
         [-0.5192,  8.7648, -3.5058,  ..., -0.5464, -1.6215,  1.6735],
         [-0.7237,  8.6417, -3.5161,  ..., -0.6146, -1.5997,  1.6597]],

        [[-0.7727,  8.4915, -3.5145,  ..., -0.6387, -1.6899,  1.7921],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         ...,
         [-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000]],

        [[-0.7727,  8.4915, -3.5145,  ..., -0.6387, -1.6899,  1.7921],
         [-0.5681,  8.6146, -3.5041,  ..., -0

In [11]:
codes.shape

torch.Size([2, 3, 462])

In [9]:
codes[:, 1, :]

tensor([[  62,   62,   62,   62,  408,  408,  408,  724,  578,  669,  620,  562,
          555,  648,  621,  778,  738,  339,  709,  985,  346,  705,  214,   86,
          975,  778,  238,  953,  574,  744,  160,  160,  574,  753,  861,  865,
          738,  705,    6,  705,  435,  560,  169,  560,  143,  237,  145,  145,
          145,  145,  254,   69,  378,  914,   74,   43, 1023,   43,  698,  857,
          328,  561,  344, 1010,   52,  106,  121,  408,  408,  408,  408,  408,
          408,  408,  408,  408,  408,  408,   62,   62,   62,   62,   62,   62,
           62,   62,   62,   62,   62,   62,   62,   62,   62,   62,   62,   62,
           62,   62,   62,   62,   62,   62,   62,   62,   62,   62,   62,  106,
          499,  499,  499,  499,  776,  147,  469,  250,  842,  757,  749,  406,
           58,  757,  250,  757,  877,  918,  913,  746,  219,  918,   24,  913,
          746,  611,  801,  908,  455,  820,  553,  553,  743,  743,  837,  203,
          280,  695,  747,  

In [5]:
audio_embed*codes_mask

tensor([[[-0.7727,  8.4915, -3.5145,  ..., -0.6387, -1.6899,  1.7921],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         ...,
         [-0.5899,  8.6741, -3.5319,  ..., -0.5725, -1.6565,  1.6399],
         [-0.5192,  8.7648, -3.5058,  ..., -0.5464, -1.6215,  1.6735],
         [-0.7237,  8.6417, -3.5161,  ..., -0.6146, -1.5997,  1.6597]],

        [[-0.7727,  8.4915, -3.5145,  ..., -0.6387, -1.6899,  1.7921],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         [-0.5681,  8.6146, -3.5041,  ..., -0.5705, -1.7116,  1.8059],
         ...,
         [-0.6388,  8.5239, -3.5303,  ..., -0.5966, -1.7466,  1.7723],
         [-0.6388,  8.5239, -3.5303,  ..., -0.5966, -1.7466,  1.7723],
         [-0.6388,  8.5239, -3.5303,  ..., -0.5966, -1.7466,  1.7723]]])

In [71]:
_, channels, input_length = inputs.input_values.shape

chunk_length = input_length
stride = input_length

step = chunk_length - stride

for offset in range(0, input_length - step, stride):
    print(offset, offset + chunk_length)

0 147620


In [50]:
res.keys()

odict_keys(['audio_codes', 'audio_scales'])

In [61]:
embeded = model.decode(res.audio_codes, res.audio_scales)

In [63]:
for frame, scale in zip(res.audio_codes, res.audio_scales):
    print("hi")

hi


In [68]:
model.config.chunk_stride

In [55]:
embeded.shape

torch.Size([2, 128, 462])

In [12]:
bandwidth = model.config.target_bandwidths[0]
embeddings = model.encoder(inputs.input_values)
codes = model.quantizer.encode(embeddings, bandwidth)
embedded = model.quantizer.decode(codes)
print(inputs.input_values.shape)
print(embeddings.shape)
print(codes.shape)
print(embedded.shape)

torch.Size([3, 1, 147620])
torch.Size([3, 128, 462])
torch.Size([2, 3, 462])
torch.Size([3, 128, 462])


In [3]:
import torch
x = torch.tensor([[[1, 2, 3, 4, 9, 8, 7, 4, 4, 4],
                   [1, 2, 4, 3, 9, 8, 7, 9, 1, 4],
                   [1, 2, 3, 4, 9, 8, 7, 9, 1, 8]]])
torch.tensor([[[1,1,1,1,1,1,1,0,0,0], [1,1,1,1,1,1,1,1,1,0], [1,1,1,1,1,1,1,1,1,1]]]).bool()*x

tensor([[[1, 2, 3, 4, 9, 8, 7, 0, 0, 0],
         [1, 2, 4, 3, 9, 8, 7, 9, 1, 0],
         [1, 2, 3, 4, 9, 8, 7, 9, 1, 8]]])

In [4]:
import torch

# Define the tensor
x = torch.tensor([[1, 2, 3, 4, 9, 8, 7, 4, 4, 4],
                   [1, 2, 4, 3, 9, 8, 7, 9, 1, 4],
                   [1, 2, 3, 4, 9, 8, 7, 9, 1, 8]])

# Check if elements are not equal to 4
not_fours = x != 4

# Reverse the tensor along the last dimension to start checking from the end
reversed_not_fours = not_fours.flip(dims=[-1])

# Use cummax to propagate the first True value encountered (from the end) across all subsequent positions
cumulative_mask = reversed_not_fours.cummax(dim=-1)[0]

# Reverse the mask back to the original order
final_mask = cumulative_mask.flip(dims=[-1])

print(final_mask.int())  # Display the mask as integers


tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.int32)


In [17]:
final_mask.unsqueeze(-1)

tensor([[[ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [False],
         [False],
         [False]],

        [[ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [False]],

        [[ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True],
         [ True]]])

In [19]:
final_mask.unsqueeze(-1).expand(-1, -1, 4).float() * x

tensor([[[0.1146, 0.9024, 0.9430, 0.0764],
         [0.2286, 0.8260, 0.3325, 0.6528],
         [0.0643, 0.7493, 0.2218, 0.1119],
         [0.7053, 0.6366, 0.7577, 0.3341],
         [0.3278, 0.3578, 0.8157, 0.2422],
         [0.9998, 0.4148, 0.9526, 0.1089],
         [0.6703, 0.8469, 0.9908, 0.6843],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.5292, 0.4454, 0.1807, 0.9610],
         [0.9582, 0.2388, 0.9461, 0.7105],
         [0.2205, 0.7982, 0.9818, 0.5728],
         [0.5858, 0.1187, 0.9630, 0.0422],
         [0.3760, 0.2717, 0.7077, 0.6838],
         [0.0281, 0.6394, 0.7935, 0.4989],
         [0.2881, 0.1807, 0.6601, 0.7565],
         [0.9185, 0.7074, 0.0926, 0.5016],
         [0.0364, 0.9290, 0.1171, 0.6538],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.1085, 0.4844, 0.2439, 0.3319],
         [0.7462, 0.7410, 0.3246, 0.8543],
         [0.1195, 0.6990, 0.3541, 0.9120],
       

In [9]:
x = torch.rand((3, 10, 4))
x

tensor([[[0.1146, 0.9024, 0.9430, 0.0764],
         [0.2286, 0.8260, 0.3325, 0.6528],
         [0.0643, 0.7493, 0.2218, 0.1119],
         [0.7053, 0.6366, 0.7577, 0.3341],
         [0.3278, 0.3578, 0.8157, 0.2422],
         [0.9998, 0.4148, 0.9526, 0.1089],
         [0.6703, 0.8469, 0.9908, 0.6843],
         [0.5544, 0.1763, 0.0176, 0.3314],
         [0.7807, 0.3720, 0.4103, 0.5085],
         [0.4169, 0.5296, 0.3217, 0.6099]],

        [[0.5292, 0.4454, 0.1807, 0.9610],
         [0.9582, 0.2388, 0.9461, 0.7105],
         [0.2205, 0.7982, 0.9818, 0.5728],
         [0.5858, 0.1187, 0.9630, 0.0422],
         [0.3760, 0.2717, 0.7077, 0.6838],
         [0.0281, 0.6394, 0.7935, 0.4989],
         [0.2881, 0.1807, 0.6601, 0.7565],
         [0.9185, 0.7074, 0.0926, 0.5016],
         [0.0364, 0.9290, 0.1171, 0.6538],
         [0.5288, 0.6977, 0.7585, 0.5893]],

        [[0.1085, 0.4844, 0.2439, 0.3319],
         [0.7462, 0.7410, 0.3246, 0.8543],
         [0.1195, 0.6990, 0.3541, 0.9120],
       

In [2]:
model.config

EncodecConfig {
  "_name_or_path": "facebook/encodec_24khz",
  "architectures": [
    "EncodecModel"
  ],
  "audio_channels": 1,
  "chunk_length_s": null,
  "codebook_dim": 128,
  "codebook_size": 1024,
  "compress": 2,
  "dilation_growth_rate": 2,
  "hidden_size": 128,
  "kernel_size": 7,
  "last_kernel_size": 7,
  "model_type": "encodec",
  "norm_type": "weight_norm",
  "normalize": false,
  "num_filters": 32,
  "num_lstm_layers": 2,
  "num_residual_layers": 1,
  "overlap": null,
  "pad_mode": "reflect",
  "residual_kernel_size": 3,
  "sampling_rate": 24000,
  "target_bandwidths": [
    1.5,
    3.0,
    6.0,
    12.0,
    24.0
  ],
  "torch_dtype": "float32",
  "transformers_version": "4.44.0",
  "trim_right_ratio": 1.0,
  "upsampling_ratios": [
    8,
    5,
    4,
    2
  ],
  "use_causal_conv": true,
  "use_conv_shortcut": true
}

In [47]:
reversed_not_fours

tensor([[[False, False, False,  True,  True,  True, False,  True,  True,  True],
         [False,  True,  True,  True,  True,  True,  True, False,  True,  True],
         [ True,  True,  True,  True,  True,  True, False,  True,  True,  True]]])

In [1]:
from utils.configs import SERConfig



In [7]:
x = SERConfig(num_labels=5)


In [8]:
x.num_labels

5

In [5]:
import numpy as np

np.array([1,1,1]).tolist()

[1, 1, 1]

In [1]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

processor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")


/Users/anhnguyenhoang/Desktop/vscode/speech-language-foundation/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import torchaudio

waveform, sampling_rate = torchaudio.load("e_happy.wav")
# print(sampling_rate)
# print(waveform.shape)
waveform = torchaudio.functional.resample(waveform, orig_freq=sampling_rate, new_freq=16000)
# print(waveform.shape)

In [2]:
processor.sampling_rate

16000

In [16]:
waveform[0].shape

torch.Size([111828])

In [18]:
inputs = processor([waveform[0].numpy(), waveform[0][:-30].numpy()], sampling_rate=16000, padding=True, return_tensors="pt", return_attention_mask=True)

In [19]:
inputs

{'input_values': tensor([[-0.0043, -0.0016, -0.0006,  ..., -0.0319,  0.1366,  0.0758],
        [-0.0044, -0.0017, -0.0006,  ...,  0.0000,  0.0000,  0.0000]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [20]:
inputs.input_values.shape

torch.Size([2, 111828])